In [27]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import loadmat
from pathlib import Path

DATA_DIR = Path('../data')

%matplotlib inline

plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 12

sns.set_style("whitegrid",{"axes.edgecolor": "0",'grid.color': '0','font.family':'Arial'})
sns.set_context("notebook",font_scale=1.5)

### Build a dictionary of the Matlab data

In [2]:
data = loadmat('../data/hije_mbon_response.mat')
x = data['Data']

ddict = {}
for i in range(17): 
    if x[0,0][i][0][0][1][0] not in ddict:
        ddict[x[0,0][i][0][0][1][0].replace("\\", '')] = x[0,0][i][0][0][0]

### Build a dataframe

Also clean the data by:
- Removing the activity in the Calyx (not part of the MB under investigation)
- Infer the name of the lobe from the MBON type
- Remove the 'empty' stimulus data

In [3]:
def get_lobe_name(mbon_type:str) -> str:
    '''Given mbon type in the format <lobe_name>xxx, return lobe_name'''
    idx_digit = re.search(r"\d", mbon_type).start()
    return mbon_type[:idx_digit + 1]

In [13]:
odors = ['vinegar', 'yeast', '2-heptanone', 'hexanol', 'octanol', 'mch', 'peppermint', 'ethanol', 'citronella', 'co2', 'empty']
raw_data = pd.DataFrame(columns = odors)

for lobe in ddict.keys():
    y = pd.DataFrame.from_dict(ddict[lobe])
    y.columns = odors
    y['lobe'] = lobe
    
    raw_data = pd.concat([raw_data, y])

# Do some cleaning
raw_data = raw_data[raw_data['lobe'] != 'calyx']
raw_data['mbon_type'] = raw_data['lobe']
raw_data['lobe'] = raw_data['lobe'].apply(get_lobe_name)
raw_data = raw_data.drop(['empty'], axis=1)

raw_data.head(10)

,vinegar,yeast,2-heptanone,hexanol,octanol,mch,peppermint,ethanol,citronella,co2,lobe,mbon_type
0,66.213579,65.091995,46.239205,52.249343,5.743157,43.539605,32.241934,43.470913,14.749717,0.301880,gamma1,gamma1pedc
1,63.139929,34.875667,53.975628,35.353596,33.534332,26.570208,33.396789,19.179342,-0.131161,-0.026777,gamma1,gamma1pedc
2,57.052937,21.330084,28.879761,64.225027,20.135808,34.136433,4.711036,12.832349,6.912816,0.414900,gamma1,gamma1pedc
3,44.990326,36.361742,41.142201,35.873428,21.418749,18.890653,33.100006,21.674654,4.949902,0.515219,gamma1,gamma1pedc
4,28.497374,45.048604,11.818865,23.184992,5.823090,13.355964,6.618679,8.484223,-2.623867,1.974842,gamma1,gamma1pedc
0,88.219581,76.250292,66.086409,52.292057,47.907946,44.267025,47.661523,44.156374,11.840836,16.626364,gamma2,gamma2alpha'1
1,58.420374,54.747254,37.821266,27.041999,13.236705,10.605523,27.826593,24.640917,8.102770,7.904836,gamma2,gamma2alpha'1
2,50.502524,47.241987,29.721496,28.210210,16.530838,14.481583,34.912318,35.123999,13.358057,15.368912,gamma2,gamma2alpha'1
3,44.635817,31.043717,31.954230,21.455287,16.420415,12.648347,23.641624,27.589108,4.648238,11.456594,gamma2,gamma2alpha'1
4,44.542182,46.427858,12.767847,20.562710,17.278679,18.691196,32.788625,23.254081,6.148264,4.022316,gamma2,gamma2alpha'1


In [24]:
# Inspect the lobes and MBON types
print(f"Lobes: {raw_data['lobe'].sort_values().unique()}")
print(f"MBON types: {raw_data['mbon_type'].sort_values().unique()}")

print('\nThe only MBON compartment with more than one documented MBON type is alpha2')
raw_data.groupby('lobe')['mbon_type'].unique().to_frame()

Lobes: ["alpha'1" "alpha'2" "alpha'3" 'alpha1' 'alpha2' 'alpha3' "beta'1"
 "beta'2" 'beta1' 'beta2' 'gamma1' 'gamma2' 'gamma3' 'gamma4' 'gamma5']
MBON types: ["alpha'1" "alpha'2" "alpha'3ap & m" 'alpha1' 'alpha2p3p' 'alpha2sc'
 'alpha3' "beta'1" "beta'2mp" 'beta1' "beta2beta'2a" 'gamma1pedc'
 "gamma2alpha'1" "gamma3 & gamma3beta'1" 'gamma4' "gamma5beta'2a"]

The only MBON compartment with more than one documented MBON type is alpha2


,mbon_type
lobe,
alpha'1,[alpha'1]
alpha'2,[alpha'2]
alpha'3,[alpha'3ap & m]
alpha1,[alpha1]
alpha2,"[alpha2p3p, alpha2sc]"
alpha3,[alpha3]
beta'1,[beta'1]
beta'2,[beta'2mp]
beta1,[beta1]


In [10]:
# stacked_data = raw_data.drop(['mbon_type'], axis=1).set_index('lobe')
# .stack()
# .to_frame().reset_index()
# stacked_data.columns = ['lobe', 'stimulus', 'dF/F']
# stacked_data.head(10)

### Calculate the mean MBON activity in each compartment/lobe

__Note__: since there were more than one MBON cell type in the alpha2 compartment, this compartment's activity is averaged over 10 trials (5 for each of the MBON cell types) 

In [25]:
mean_data = raw_data.groupby('lobe').mean()
mean_data

,vinegar,yeast,2-heptanone,hexanol,octanol,mch,peppermint,ethanol,citronella,co2
lobe,,,,,,,,,,
alpha'1,42.207852,41.415112,37.345698,21.356445,23.697758,17.463519,23.121837,21.848225,6.673281,6.619625
alpha'2,67.893369,65.921205,56.757224,60.258264,44.895844,40.968825,55.030486,25.903130,16.205035,12.677670
alpha'3,19.891953,17.172300,16.527422,12.791175,11.418638,14.518355,15.331365,11.196726,6.235885,6.132741
alpha1,18.224721,15.204187,25.195775,21.534141,15.786560,14.516087,11.604648,18.995279,6.950629,4.534904
alpha2,22.061204,20.095421,26.507974,22.699176,16.139411,11.506731,8.603303,13.796766,5.896193,9.346885
alpha3,65.533156,73.104286,59.853255,57.371698,40.676845,38.627667,50.154443,39.737071,20.897464,9.005112
beta'1,54.377704,53.161709,9.596619,4.755605,8.427114,6.039949,19.877868,19.247783,6.953496,9.320091
beta'2,19.122448,18.998279,26.951265,27.698609,22.528744,20.891894,19.256622,12.367632,13.379730,9.404558
beta1,35.949324,24.086183,33.091999,30.181439,22.546595,17.944532,20.709696,18.581403,8.588297,4.898470


In [ ]:
# Plot a subset
mean_data[['vinegar', 'yeast', 'hexanol']].plot.bar(rot=90)

In [ ]:
corr = mean_data.T.corr()

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)

corr

In [28]:
mean_data.to_csv(DATA_DIR / 'hije_pivot_table.csv')